# Training Pipeline

**SageMaker Studio Kernel**: Data Science

In this exercise you will do:
 - Create/Run an Amazon SageMaker Pipeline [SageMaker Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines.html)

***

## Part 1/3 - Setup
Here we'll import some libraries and define some variables.

In [ ]:
import boto3
import json
import logging
import os
import sagemaker
import sys

In [ ]:
sys.path.insert(0, os.path.abspath('./../mlpipelines'))

In [ ]:
from training.pipeline import get_pipeline

In [ ]:
s3_client = boto3.client('s3')
sm_client = boto3.client('sagemaker')

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

***

## Part 2/3 - Create Amazon SageMaker Pipeline

### Explore Pipeline definition

In [ ]:
!pygmentize ./../mlpipelines/training/pipeline.py

### Pipeline Parameters

In [ ]:
region = boto3.session.Session().region_name
role_name = "mlops-sagemaker-execution-role"
role = "arn:aws:iam::{}:role/{}".format(boto3.client('sts').get_caller_identity().get('Account'), role_name)

kms_account_id = boto3.client('sts').get_caller_identity().get('Account')
kms_alias = "ml-kms"

bucket_name = ""

inference_instance_type = "ml.m5.xlarge"

model_package_group_name = "ml-end-to-end-group"
model_approval_status = "PendingManualApproval"

processing_artifact_path = "artifact/processing"
processing_artifact_name = "sourcedir.tar.gz"
processing_framework_version = "0.23-1"
processing_instance_count = 1
processing_instance_type = "ml.t3.large"
processing_input_files_path = "data/input"
processing_output_files_path = "data/output"

training_artifact_path = "artifact/training"
training_artifact_name = "sourcedir.tar.gz"
training_output_files_path = "models"
training_framework_version = "2.4"
training_python_version = "py37"
training_instance_count = 1
training_instance_type = "ml.p2.xlarge"
training_hyperparameters = {
    "epochs": 5,
    "learning_rate": 1.45e-4,
    "batch_size": 100
}

### Compress source code for installing additional python modules

In [ ]:
! ./../algorithms/buildspec.sh training $bucket_name

In [ ]:
! ./../algorithms/buildspec.sh processing $bucket_name

### Get the dataset and upload it to an S3 bucket

In [ ]:
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client("sagemaker")
runtime_client = boto_session.client("sagemaker-runtime")

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=runtime_client,
    default_bucket=bucket_name
)

In [ ]:
# Download the 
# clean the buckets first
s3_client.delete_object(Bucket=bucket_name, Key=processing_input_files_path)

input_data = sagemaker_session.upload_data('./../data/data.csv', key_prefix=processing_input_files_path)

print(input_data)

***

## Training pipeline

### Get pipeline definition

In [ ]:
pipeline = get_pipeline(
    region,
    kms_account_id,
    kms_alias,
    bucket_name,
    inference_instance_type,
    model_package_group_name,
    processing_artifact_path,
    processing_artifact_name,
    processing_framework_version,
    processing_instance_count,
    processing_instance_type,
    processing_input_files_path,
    processing_output_files_path,
    training_artifact_path,
    training_artifact_name,
    training_output_files_path,
    training_framework_version,
    training_python_version,
    training_instance_count,
    training_instance_type,
    training_hyperparameters,
    role=role,
    pipeline_name="MLOpsTrainPipeline"
)

### Create or update SageMaker pipeline

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
json.loads(pipeline.definition())

***

## Part 3/3 - Run SageMaker Pipeline

### Start training pipeline 

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()

### Start training pipeline and overriding parameters

In [ ]:
args = {
#    "Epochs": "",
#    "ModelApprovalStatus": "",
#    "ModelPackageGroupName": "",
    "ProcessingInput": "s3://{}/{}".format(bucket_name, processing_input_files_path)
}

In [ ]:
execution = pipeline.start(
    parameters=args
)

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()